# Rule Unit 작성 예시

본 문서는 1차년도 6월에 개발된 RuleUnit 클래스를 활용하여 룰 유닛을 작성하고 실행하는 과정을 설명합니다.

⚠︎ 주의사항: 코드를 변경하며 작업할 때는, 사본을 생성한 후 작업해주십시오.

## 룰 유닛 작성하기
이 부분에서는 기존의 IFC 파일에 종속적이었던 실행 흐름에서 벗어나, 새로이 정의한 RuleUnit 구조에 기반하여 룰에 포함된 항목을 작성하는 방법을 다룹니다.<br><br>
본 예시에서는 KDS 24 14 21 4.6.5.1(5) 를 룰 유닛으로 작성합니다.

먼저, Rule 작성에 필요한 프로그램을 불러오기 위해 Github에서 아래의 주소가 가리키는 파일 모음을 불러옵니다. (Github Repository를 clone합니다.)

In [1]:
!git clone https://github.com/KU-HIAI/tomok.git

Cloning into 'tomok'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 175 (delta 54), reused 85 (delta 31), pack-reused 58
Receiving objects: 100% (175/175), 451.83 KiB | 2.24 MiB/s, done.
Resolving deltas: 100% (62/62), done.


필요한 파일을 불러온 다음에는 현재 작업 경로를 tomok의 파일이 있는 위치로 이동합니다.<br><br>
아래의 코드 블록을 실행하면, 작성자의 기본 작업 경로가 필요한 파일이 있는 위치로 이동됩니다.

In [2]:
cd tomok

/content/tomok


RuleUnit의 실행 함수는 기존의 OKNGResult 클래스를 활용하지 않고, 해당 실행함수의 기준을 만족하였는지의 여부를 bool 타입의 값(참/거짓)으로 반환합니다.

## 룰 유닛 작성해보기
지금까지의 내용을 바탕으로 룰 유닛을 작성해봅시다.

새로운 룰 유닛을 작성하기 위해서는, 아래 코드 블럭에서 클래스 이름, 메타정보에 해당하는 변수, 실행 함수의 이름과 내용을 수정하면 됩니다.

In [3]:
import math
from typing import List

import tomok

# 작성하는 룰에 맞게 클래스 이름 수정 (KDS241711_04060303_06)
class KDS241711_04060303_06(tomok.RuleUnit): # KDS241711_04060303_06

    # 아래 클래스 멤버 변수에 할당되는 값들을 작성하는 룰에 맞게 수정
    priority = 1   # 건설기준 우선순위
    author = 'Chanwoo Yang'  # 작성자명
    ref_code = 'KDS 24 17 11 4.6.3.3 (6)' # 건설기준문서
    ref_date = '2022-02-25'  # 디지털 건설문서 작성일  (고시일)
    doc_date = '2023-11-14'  # 건설기준문서->디지털 건설기준 변환 기준일 (작성 년월)
    title = '축방향철근과 횡방향철근'   # 건설기준명

    # 건설기준문서항목 (분류체계정보)
    description = """
    교량내진설계기준(한계상태설계법)
    4. 설계
    4.6 콘크리트교 설계
    4.6.3 기둥
    4.6.3.3 축방향철근과 횡방향철근
    (6)
    """
    # https://dillinger.io/ 표와 이미지 랜더링 확인 사이트
    # 이미지 링크 변환 사이트 https://www.somanet.xyz/2017/06/blog-post_21.html
    # 건설기준문서내용(text)
    content = """
    #### 4.6.3.3 축방향철근과 횡방향철근
    \n (6) 심부구속 횡방향철근과 단부구역의 횡방향철근은 인접부재와의 연결면으로부터 기둥 치수의 0.5배와 380mm 중 큰 값 이상까지 연장해서 설치하여야 한다.
    """

    # 플로우차트(mermaid)
    flowchart = """
    flowchart TD
	  subgraph Python_Class
	  A([축방향철근과 횡방향철근])
	  B["KDS 24 17 11 4.6.3.3(6)"]
	  A ~~~ B
	  end

	  subgraph Variable_def
	  VarIn[/입력변수: 기둥치수/]
	  end
	  Python_Class ~~~ Variable_def---> D
	  D["심부구속 횡방향철근과 단부구역의 횡방향철근 설치 ≥ max(기둥치수 x0.5, 380mm)"]
	  E(["PASS or Fail"])

	  D-->E
    """

    # 작성하는 룰에 맞게 함수 이름과 내용을 수정
    @tomok.rule_method
    def additional_length_of_horizontal_rebar(fIadlehore,fIcolsiz) -> bool:
        """횡방향철근의 연장길이

        Args:
            fIadlehr (float): 횡방향철근의 연장길이
            fIcolsiz (float): 기둥치수

        Returns:
            bool: 교량내진설계기준(한계상태설계법) 4.6.3.3 축방향철근과 횡방향철근 (6)의 통과 여부
        """

        if fIadlehr >= max(0.5 * fIcolsiz, 380):
          return "Pass"
        else:
          return "Fail"

In [4]:
my_RuleUnit = KDS241711_04060303_06()

In [5]:
fIadlehr = 600
fIcolsiz = 1000

In [6]:
Rule_Review_Result = my_RuleUnit.additional_length_of_horizontal_rebar(fIadlehr,fIcolsiz)
print("RuleUnit Review Result: {}".format(Rule_Review_Result))

RuleUnit Review Result: Pass


<br><br>
아래의 코드를 통해, 룰 유닛의 content(건설기준 항목의 실제 내용)의 markdown 렌더링 결과를 확인할 수 있습니다.

In [7]:
my_RuleUnit.render_markdown()


    #### 4.6.3.3 축방향철근과 횡방향철근
    
 (6) 심부구속 횡방향철근과 단부구역의 횡방향철근은 인접부재와의 연결면으로부터 기둥 치수의 0.5배와 380mm 중 큰 값 이상까지 연장해서 설치하여야 한다.
    